In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.agent import AgentFinish
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from pydantic import BaseModel, Field
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from pydantic import BaseModel, Field

In [2]:
openai_llm = ChatOpenAI(temperature=0, model="gpt-4o")

C:\Users\Design Dep\AppData\Local\Temp\ipykernel_8616\1305393998.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  openai_llm = ChatOpenAI(temperature=0, model="gpt-4o")


In [13]:
chat_template ="""

You are an expert in analyzing CVs and providing personalized career advice. Given an individual's CV, you will:

1. Conduct a detailed analysis of their skills, experience, qualifications, and overall suitability for various roles.
2. Recommend the best job opportunities that align with their profile, ensuring that the suggestions are well-matched to their strengths and career goals.
3. Take into account current industry trends, job market demand, and emerging opportunities to ensure that your recommendations are both relevant and forward-thinking.
4. Provide actionable insights that can help improve the CV, such as highlighting areas for growth or skill enhancement based on market needs.

Your goal is to give comprehensive, data-driven advice that helps individuals make informed career decisions.

below is the CV:
{input}


The output should have the following keywords

Skills_and_experience_analysis,
job_opportunities
industry_trends_and_market_demand
cv_improvement_suggestions
"""



In [20]:
import gradio as gr
import os
def get_file(file_path):
    directory_name = os.path.dirname(file_path)  
    
    documents = SimpleDirectoryReader(directory_name).load_data()    
    prompt = ChatPromptTemplate.from_template(chat_template)
    chain = LLMChain(prompt=prompt, llm=openai_llm)
    global doc_info 
    doc_info = documents[0].text
    response = chain.run({"input":doc_info})
    return response
demo = gr.Interface(fn=get_file, inputs=gr.File(), outputs="text")


In [21]:
demo.launch(share=True)
# demo.close()

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://c046452bebc784c052.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
class CVDetails(BaseModel):
    cv_summary :str = Field(description="The overall summary of the person Cv")

In [ ]:
from bs4 import BeautifulSoup
import requests

html_text = requests.get("https://www.myjob.mu/Jobs/ICT-IT-Web/").text

soup = BeautifulSoup(html_text, "lxml")
job_results = soup.find_all("div", class_="module job-result")
if len(job_results)> 0:
    for job in job_results:
        company_name = job.find('h3', itemprop="name")
        company_name = company_name.a.text
        print(f"company_name is {company_name}")
        job_name = job.find("h2", itemprop="title")
        job_name = job_name.a.text
        job_results_overview = job.find("div", class_="job-result-overview")
        overview = job_results_overview.find("")
        print(f"job name is: {job_name}")
        #job_description =

